In [4]:
Get-Help -examples Publish-RsProject


NAME
    Publish-RsProject
    
SYNOPSIS
    This script deploys a Reporting Services project to a Power BI Report Server.
    
    
    -------------------------- EXAMPLE 1 --------------------------
    
    PS C:\>Get-RsDeploymentConfig -ProjectFile 'C:\Users\Aaron\source\repos\Finance\Financial 
    Reports\SSRS_FR\SSRS_FR.rptproj' |
    
    Add-Member -PassThru -MemberType NoteProperty -Name ReportPortal -Value 'http://localhost/PBIRSportal/' |
    Publish-RsProject
    
    Description
    -----------
    Get-RsDeploymentConfig prompts the user to select which deployment configuration to use from 
    the 'C:\Users\Aaron\source\repos\Finance\Financial Reports\SSRS_FR\SSRS_FR.rptproj' project file.  These settings 
    are piped to the Add-Member where the ReportPortal property is added and set to 'http://localhost/PBIRSportal/'.
    The settings are then piped to the Publish-RsProject function, which deploys all project files using all 
    applicable 
    settings from the dep

In [5]:
Get-RsDeploymentConfig -RsProjectFile 'C:\temp\SQLServerPerformanceDashboardReportingSolution\SQL Server Performance Dashboard\SQL Server Performance Dashboard.rptproj'



FullPath               : DebugLocal
OverwriteDatasets      : False
OverwriteDataSources   : False
TargetReportFolder     : SQL Server Performance Dashboard
TargetDatasetFolder    : Datasets
TargetDatasourceFolder : Data Sources
TargetReportPartFolder : Report Parts
TargetServerURL        : 
RsProjectFolder        : C:\temp\SQLServerPerformanceDashboardReportingSolution\SQL Server Performance Dashboard





When we grab the config file, we'll need to add a property for the Report Portal URL, because the <span style="color: rgb(139, 0, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">.rptproj</span> file just doesn't include that info.

In [16]:
$RSConfig = Get-RsDeploymentConfig -ConfigurationToUse Release -RsProjectFile 'C:\temp\SQLServerPerformanceDashboardReportingSolution\SQL Server Performance Dashboard\SQL Server Performance Dashboard.rptproj' |
Add-Member -PassThru -MemberType NoteProperty -Name ReportPortal -Value 'http://localhost/Reports/'

Let's just take a look at the config, before we try to deploy anything

In [17]:
$RSConfig



FullPath               : Release
OverwriteDatasets      : False
OverwriteDataSources   : False
TargetReportFolder     : SQL Server Performance Dashboard
TargetDatasetFolder    : Datasets
TargetDatasourceFolder : Data Sources
TargetReportPartFolder : Report Parts
TargetServerURL        : http://localhost/ReportServer/
RsProjectFolder        : C:\temp\SQLServerPerformanceDashboardReportingSolution\SQL Server Performance Dashboard
ReportPortal           : http://localhost/Reports/





In [28]:
$RSConfig.TargetServerURL = 'http://localhost/ReportServer/'
$RSConfig.TargetReportFolder = '/CDW Insights'

In [25]:
$RSConfig | Publish-RsProject

$RsProjectFolder = 'C:\temp\SQLServerPerformanceDashboardReportingSolution\SQL Server Performance Dashboard' being used is
    This deployment is going to happen using the following settings...
    


FullPath               : Release
OverwriteDatasets      : False
OverwriteDataSources   : False
TargetReportFolder     : CDW Insights
TargetDatasetFolder    : Datasets
TargetDatasourceFolder : Data Sources
TargetReportPartFolder : Report Parts
TargetServerURL        : http://localhost/ReportServer/
RsProjectFolder        : C:\temp\SQLServerPerformanceDashboardReportingSolution\SQL Server Performance Dashboard
ReportPortal           : http://localhost/Reports/




    Beginning deployment.
    Building folder structures...
    


VERBOSE: Making call to http://localhost/Reports/api/v2.0/me to create a session...


VERBOSE: Reading XSRF Token cookie...
VERBOSE: Decoding XSRF Token and setting it as a header of the session...
VERBOSE: Creating folder /CDW Insights...


VERBOSE: Folder /CDW Insights was created successfully!


@odata.context : http://localhost/Reports/api/v2.0/$metadata#Folders/$entity
Id             : c53d8b7b-51c0-4683-952d-b655db62d2fb
Name           : CDW Insights
Description    : 
Path           : /CDW Insights
Type           : Folder
Hidden         : False
Size           : 0
ModifiedBy     : NORTHAMERICA\aanelson
ModifiedDate   : 2021-04-28T10:09:14.927-04:00
CreatedBy      : NORTHAMERICA\aanelson
CreatedDate    : 2021-04-28T10:09:14.927-04:00
ParentFolderId : 39c89991-7adf-4232-8840-21346ecd101b
IsFavorite     : False
ContentType    : 
Content        : 
Roles          : {}



Type   Name         Size CreatedBy             CreatedDate                  ModifiedBy            ModifiedDate         
----   ----         ---- ---------             -----------                  ----------            ------------         
Folder Data Sources    0 NORTHAMERICA\aanelson 2020-11-02T13:28:11.82-05:00 NORTHAMERICA\aanelson 2020-11-02T13:28:1...




...

wait_blocking


wait_buffer_io


wait_buflatch


wait_generic


wait_latch


Take a look and make sure all of our reports got deployed

In [29]:
Get-RsFolderContent -ReportServerUri $RSConfig.TargetServerURL -RsFolder $RSConfig.TargetReportFolder |
Format-Table


ID                                   Name                        Path                                      VirtualPath
--                                   ----                        ----                                      -----------
09245381-0b5a-41f1-874e-fd50b5de644d database_overview           /CDW Insights/database_overview                      
02c696eb-59bd-4f31-9760-00cf52f05fc6 database_storage_report     /CDW Insights/database_storage_report                
5eeebf2f-3259-49c0-b76e-2c4a140933d1 historical_io               /CDW Insights/historical_io                          
01a72287-2d85-412f-8a7f-ab03c17aa7de historical_waits            /CDW Insights/historical_waits                       
cdd56cfc-71b9-4ac4-a64e-d9301fbcbadf missing_index_from_showplan /CDW Insights/missing_index_from_showplan            
bf9defcd-6d9c-4386-b3c3-4cf70acf33f9 missing_indexes             /CDW Insights/missing_indexes                        
7231f330-2d8a-4bc9-a3b1-141342c09d0e page_detai